In [ ]:
def getBetaParams(mus,sigma2s):
    '''
    Converts the mean and variance to a and b.
    Only valid if sigma2 <= mu(1-mu)
    '''
    assert np.all(sigma2s <= mus*(1-mus))
    num = (sigma2s + mus**2 - mus)/sigma2s
    a = -1*mus*num
    b = num*(mus - 1)
    assert np.all(a > 0) or np.all(b > 0), 'Consider lowering noise levels'
    return a,b

In [ ]:
def addNoise(df,colname,output_colname,y_noise):
    '''
    Add noise to the deltas. Models the noisy output as a sample from
    a Beta distribution. y_noise is the standard deviation around the mean df[colname]
    '''
    y = df[colname]
    a,b = getBetaParams(y,y_noise**2)
    y_mod = beta(a,b).rvs(len(y))
    y_mod = np.round(y_mod,3)
    df[output_colname] = y_mod
    return df

In [ ]:
def estimateCoeffs(df,alpha_cols):
    x = df[alpha_cols]
    y = df['y'].values
    md = LinearRegression(fit_intercept=False)
    md.fit(x,y)
    # calculate error
    yhat = md.predict(x)
    sigma2 = np.sum((yhat - y)**2)/(df.shape[0]-x.shape[1])
    # calculate variance of estimate
    var = np.linalg.inv(np.dot(x.T,x))*sigma2
    return md.coef_,var

In [ ]:
def estimateCoeffsBayesian(df,alpha_cols):
    x = df[alpha_cols]
    y = df['y'].values
    md = LinearRegression(fit_intercept=False)
    md.fit(x,y)
    # calculate error
    yhat = md.predict(x)
    sigma2 = np.sum((yhat - y)**2)/(df.shape[0]-x.shape[1])
    # calculate variance of estimate
    var = np.linalg.inv(np.dot(x.T,x))*sigma2
    return md.coef_,var

In [ ]:
def impSampFast(df,alphacols,qparams,pparams,qvar,pvar,T=10000):
    # run importance sampling
    # set up the q distribution
    # the coef qvar helps it budge from the q distribution - the var from the least square estimate can help us guide if we 
    # need to budge far away from the information- if more, then we don't need to budge; if less, then budge
    assert T > 1, 'T has to be greater than 1'
    qdiric = dirichlet(qvar*qparams)
    pdiric = dirichlet(pvar*pparams)
    draws = qdiric.rvs(T)
    logqvals = qdiric.logpdf(draws.T)
    logpriors = pdiric.logpdf(draws.T)

    x = df[alphacols].values
    y = df['y'].values
    xb_all = np.dot(x,draws.T)
    mus_all = np.exp(xb_all)/(1 + np.exp(xb_all))
    phis = np.repeat(df['y_var'].values[:,np.newaxis],T,axis=1)
    a_all,b_all = getBetaParams(mus_all,phis)
    logliks = beta(a_all,b_all).logpdf(y[:,np.newaxis]).sum(axis=0)
    logpvals = logpriors + logliks
    logwts = logpvals - logqvals
    wts = np.exp(logwts)
    
    draws = np.array(draws).squeeze()
    imp_samp_df = pd.DataFrame(np.vstack([np.array(logpriors),np.array(logliks),
                                          np.array(logqvals),np.array(logpvals),np.array(wts)]).T,
                           columns=['logprior','loglik','logqval','logpval','wt'])
    imp_samp_df['wt_norm'] = imp_samp_df['wt']/imp_samp_df['wt'].sum()
    
    #imp_samp_df.dropna(inplace=True)
    #assert imp_samp_df.shape[0] > 0, 'No samples were close to target distribution, consider reducing (phi) noise levels'
    #  self-normalized importance sampling estimate
    post_exp_value = np.sum(draws*wts[:,np.newaxis],axis=0)/np.sum(wts)

    #  self-normalized importance variance estimate
    wts_norm = wts/np.sum(wts)
    post_var = np.sum((wts_norm[:,np.newaxis]**2)*(draws - post_exp_value)**2,axis=0)
    
    return post_exp_value,post_var,imp_samp_df

In [ ]:
# this function calculates the pro support for a topic on sm via sensing
def runSimFast(N,sm_dist,survey_x,ideol_dist,part_bias):
    '''
    N         - number of sm users for a given topic for sensing
    sm_dist   - demographic breakup on that sm
    survey_x  - response rates for demographic subgroups from surveys
                broken-up by ideologies
    ideol_dist- probability indicating the ideological distribution
    part_bias - participation bias induced by ideology
                (scalar value between -1 and +1)
                
    The ideologies are denoted as ideol0 and ideol1. A positive part_bias
    indicates that ideol1 is more likely to participate and negative
    part_bias indicates that ideol0 is more likely to participate.
    '''
    # precompute the participation probability for each ideology
    
    if part_bias > 0:
        part_probs = {'ideol1':1, 'ideol0':1 - part_bias}
        
    if part_bias < 0:
        part_probs = {'ideol1':1 + part_bias, 'ideol0':1}
    
    if part_bias == 0:
        part_probs = {'ideol1':1 , 'ideol0':1}
    # picking a particular demographic group for N users. 
    # dems is a matrix of N X d, and each row will have a single 1
    dems = multinomial(1,np.array(list(sm_dist.values()))).rvs(N)
    dem_names = {}
    for i,k in enumerate(sm_dist.keys()):
        dem_names[i] = k

    # picking ideology for N users.
    ideols = bernoulli(ideol_dist).rvs(N)
    dem_inds = np.where(dems == 1)[1] + 1

    ideols_df = pd.DataFrame(ideols,columns=['ideol'])
    ideols_df['ideol'] = ideols_df['ideol'].astype(str)
    ideols_df['ideol_st'] = 'ideol'+ideols_df['ideol']

    part_probs_list = ideols_df['ideol_st'].map(part_probs).values
    part_flags = bernoulli(part_probs_list).rvs(N)

    ideols_df['dem'] = dem_inds
    ideols_df['dem'] = ideols_df['dem'].astype(str)
    ideols_df['dem_st'] = 'dem'+ideols_df['dem']

    ideols_df['f_dem_st'] = 'x_' + ideols_df['dem_st'] + '_'+ideols_df['ideol_st']
    ideols_df['part_flag'] = part_flags

    
    dem_resp_rate = ideols_df['f_dem_st'].map(survey_x)
    resp = bernoulli(dem_resp_rate).rvs(N)
    ideols_df['response'] = resp
    responses_sm = resp[np.where(part_flags == 1)[0]]
    y_dem=len(np.where(responses_sm == 1)[0])/len(responses_sm)
    return y_dem, ideols_df

In [ ]:
# MEAN AVERAGE PERCENTAGE ERROR (MAPE)
def normmae(actual,predicted,norm=True):
    if norm:
        return np.mean(np.abs((actual - predicted)/actual))
    else:
        return np.mean(np.abs((actual - predicted)))

In [ ]:
def simSM(df_survey_dem_x,num_users,sm_dem_dist,ideol_dist,var_prior,noise_level,
          part_bias_diverse=True,part_bias=0):
    dem_y = {}
    true_dem_dist = {}
    if part_bias_diverse:
        part_bias_dist = beta(1,1)
    part_biases = []
    # iterates over surveys
    for t in df_survey_dem_x.index:
        if part_bias_diverse:
            part_bias = 2*(0.5-part_bias_dist.rvs(1))[0]
        part_biases.append(part_bias)
        _dem_y_t,responses_sm = runSimFast(num_users,sm_dem_dist,
                              df_survey_dem_x.loc[t].to_dict(),
                              ideol_dist,part_bias)
        dem_y[t] = _dem_y_t
        _true_dem_dist = getTrueDemDist(responses_sm)
        true_dem_dist[t] = _true_dem_dist
        

    # convert to a DataFrame
    dem_y_dict = {'y_true':dem_y}
    true_dem_dict = pd.DataFrame.from_dict(true_dem_dist,
                                 orient='index')
    true_dem_dict.columns = ['true_'+_c[2:] for _c in df_survey_dem_x.columns]
    df = df_survey_dem_x.join(pd.DataFrame.from_dict(dem_y_dict)).join(true_dem_dict)
    # add noise and sample variance
    df = addNoise(df,'y_true','y',noise_level*np.ones((df.shape[0],)))
    invg = invgamma(1/var_prior)
    df['y_var'] = invg.rvs(df.shape[0])
    return df,part_biases

In [ ]:
# estimate true demographic distribution of participants
def getTrueDemDist(responses_sm):
    responses_sm_p = responses_sm.loc[responses_sm['part_flag'] == 1]
    return (responses_sm_p.groupby('f_dem_st').size()/responses_sm_p.shape[0]).to_dict()

In [ ]:
# run inference experiments
def runInfExptFixed(df,prior_sm_dem_dist,qvar,pvar,alphacols,pprior=[]):    
    w,var = estimateCoeffs(df,alphacols)
    wnorm = w.copy()
    wnorm[np.where(wnorm <= 0)[0]] = 1e-4
    if len(pprior) == 0:
        pparams = wnorm
    else:
        pparams = pprior
    qparams = prior_sm_dem_dist
    post_w,post_var,imp_samp_df = impSampFast(df,
                                          alphacols,
                                          qparams,pparams,
                                          qvar=qvar,pvar=pvar)
    return w,var,post_w,post_var


In [ ]:
# Calculate errors
def calcErrors(est_ws,est_vars,est_post_ws,est_post_vars,fact_sm_dem_dist):
    est_errors = []
    est_pes = []
    for w,var in zip(est_ws,est_vars):
        est_errors.append(np.abs(w - fact_sm_dem_dist))
        est_pes.append(np.abs((w - fact_sm_dem_dist)/fact_sm_dem_dist))
    est_errors = np.array(est_errors)
    est_pes = np.array(est_pes)

    est_post_errors = []
    est_post_pes = []
    for w,var in zip(est_post_ws,est_post_vars):
        est_post_errors.append(np.abs(w - fact_sm_dem_dist))
        est_post_pes.append(np.abs((w - fact_sm_dem_dist)/fact_sm_dem_dist))
    est_post_errors = np.array(est_post_errors)
    est_post_pes = np.array(est_post_pes)

    res = {}
    res['truth'] = fact_sm_dem_dist
    res['ls_mean'] = np.array(est_ws).mean(axis=0)
    res['ls_mean_std'] = np.std(np.array(est_ws),axis=0)
    res['ls_var'] = np.array(est_vars).mean(axis=0)
    res['ls_errors'] = np.array(est_errors).mean(axis=0)
    res['ls_pes'] = np.array(est_pes).mean(axis=0)

    res['is_mean'] = np.array(est_post_ws).mean(axis=0)
    res['is_mean_std'] = np.std(np.array(est_post_ws),axis=0)
    res['is_var'] = np.array(est_post_vars).mean(axis=0)
    res['is_errors'] = np.array(est_post_errors).mean(axis=0)
    res['is_pes'] = np.array(est_post_pes).mean(axis=0)
    
    return res
